In [1]:
%%pyspark
df = spark.read.load('abfss://wwi-02@asadatalakeyv8r2e0.dfs.core.windows.net/sale-small/Year=2019/Quarter=Q4/Month=12/Day=20191231/sale-small-20191231-snappy.parquet', format='parquet')
display(df.limit(10))

datalake='asadatalakeyv8r2e0'

StatementMeta(SparkPool01, 7, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3333f942-27be-40fa-b883-bbf90fbca8d2)

In [2]:
df.printSchema()

StatementMeta(SparkPool01, 7, 3, Finished, Available)

root
 |-- TransactionId: string (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- ProductId: short (nullable = true)
 |-- Quantity: byte (nullable = true)
 |-- Price: decimal(38,18) (nullable = true)
 |-- TotalAmount: decimal(38,18) (nullable = true)
 |-- TransactionDate: integer (nullable = true)
 |-- ProfitAmount: decimal(38,18) (nullable = true)
 |-- Hour: byte (nullable = true)
 |-- Minute: byte (nullable = true)
 |-- StoreId: short (nullable = true)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

profitByDateProduct = (df.groupBy("TransactionDate","ProductId")
    .agg(
        sum("ProfitAmount").alias("(sum)ProfitAmount"),
        round(avg("Quantity"), 4).alias("(avg)Quantity"),
        sum("Quantity").alias("(sum)Quantity"))
    .orderBy("TransactionDate"))
display(profitByDateProduct.limit(100))

StatementMeta(SparkPool01, 7, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, a57490bb-7fd8-4aa6-adb6-1ee65fe792f3)

In [1]:
%%pyspark
dfjson = spark.read.load('abfss://datalakecsv@asadatalakeyv8r2e0.dfs.core.windows.net/00-asa-workspace-core.json', format='json',multiLine=True, inferSchema=True)
display(dfjson.limit(10))

StatementMeta(SparkPool01, 8, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 1d7d43b9-776e-416c-9a1f-7948f763b74d)

In [2]:
dfjson1 = (spark.read \
        .option('inferSchema', 'true') \
        .json('abfss://datalakecsv@asadatalakeyv8r2e0.dfs.core.windows.net/00-asa-workspace-core.json', multiLine=True)
    )

display(dfjson1.limit(10))

StatementMeta(SparkPool01, 8, 3, Finished, Available)

SynapseWidget(Synapse.DataFrame, 79c2dfb1-7cd2-46b4-8393-16dd23b83de0)

In [ ]:
dfjsonFlat=dfjson.select(explore('resources').alias('resourcesFlat'))
dfjsonFlat.show()

In [5]:
from pyspark.sql.functions import udf, explode

dfjsonTypeName=(dfjson.select(explode('resources').alias('resourcesFlat'))
.select('resourcesFlat.type','resourcesFlat.name')
)
dfjsonTypeName.show()


StatementMeta(SparkPool01, 8, 6, Finished, Available)

+--------------------+--------------------+
|                type|                name|
+--------------------+--------------------+
|Microsoft.Resourc...|             default|
|Microsoft.Storage...|[variables('blobS...|
|Microsoft.Storage...|[concat(variables...|
|Microsoft.KeyVaul...|[variables('keyVa...|
|Microsoft.KeyVaul...|[concat(variables...|
|Microsoft.Storage...|[variables('adlsS...|
|Microsoft.Storage...|[concat(variables...|
|Microsoft.Storage...|[concat(variables...|
|Microsoft.Synapse...|[variables('works...|
|Microsoft.Storage...|[concat(variables...|
|Microsoft.Authori...|[guid(concat(reso...|
|Microsoft.Synapse...|[concat(variables...|
|Microsoft.Storage...|[concat(variables...|
|Microsoft.Storage...|[concat(variables...|
|Microsoft.Storage...|[concat(variables...|
|Microsoft.Documen...|[variables('cosmo...|
|Microsoft.Documen...|[concat(variables...|
|Microsoft.Documen...|[concat(variables...|
|Microsoft.Documen...|[concat(variables...|
|Microsoft.EventHu...|[variables

In [7]:
datalake='asadatalakeyv8r2e0'
df = (spark.read \
        .option('inferSchema', 'true') \
        .json('abfss://wwi-02@' + datalake + '.dfs.core.windows.net/online-user-profiles-02/*.json', multiLine=True)
    )

df.printSchema()

StatementMeta(SparkPool01, 8, 8, Finished, Available)

root
 |-- topProductPurchases: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- itemsPurchasedLast12Months: long (nullable = true)
 |    |    |-- productId: long (nullable = true)
 |-- visitorId: long (nullable = true)

In [8]:
# create a view called user_profiles
df.createOrReplaceTempView("user_profiles")

StatementMeta(SparkPool01, 8, 9, Finished, Available)

In [9]:
from pyspark.sql.functions import udf, explode

flat=df.select('visitorId',explode('topProductPurchases').alias('topProductPurchases_flat'))
flat.show(100)

StatementMeta(SparkPool01, 8, 10, Finished, Available)

+---------+------------------------+
|visitorId|topProductPurchases_flat|
+---------+------------------------+
|   117000|              [13, 3623]|
|   117000|               [5, 2321]|
|   117001|               [93, 713]|
|   117001|              [19, 2144]|
|   117001|              [30, 1094]|
|   117001|              [82, 3223]|
|   117001|              [42, 3328]|
|   117001|              [62, 2926]|
|   117001|              [63, 2651]|
|   117001|               [39, 341]|
|   117001|              [85, 4841]|
|   117001|              [67, 4289]|
|   117001|              [42, 1264]|
|   117001|              [43, 3608]|
|   117001|               [14, 504]|
|   117001|              [97, 2649]|
|   117001|              [44, 2873]|
|   117001|               [7, 4491]|
|   117001|              [70, 3623]|
|   117001|               [4, 4035]|
|   117001|              [45, 3576]|
|   117002|               [25, 478]|
|   117002|              [15, 2515]|
|   117002|               [34, 209]|
|

In [10]:
topPurchases = (flat.select('visitorId','topProductPurchases_flat.productId','topProductPurchases_flat.itemsPurchasedLast12Months')
    .orderBy('visitorId'))

# topPurchases.show(100)

StatementMeta(SparkPool01, 8, 11, Finished, Available)

In [11]:
sortedTopPurchases = topPurchases.orderBy("itemsPurchasedLast12Months")

StatementMeta(SparkPool01, 8, 12, Finished, Available)

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

sortedTopPurchases = (topPurchases
    .orderBy( col("itemsPurchasedLast12Months").desc() ))

StatementMeta(SparkPool01, 8, 14, Finished, Available)

In [14]:
groupedTopPurchases = (sortedTopPurchases.select("visitorId")
    .groupBy("visitorId")
    .agg(count("*").alias("total"))
    .orderBy("visitorId") )

# display(groupedTopPurchases.limit(100))

StatementMeta(SparkPool01, 8, 15, Finished, Available)

In [15]:
display(groupedTopPurchases.limit(100))

StatementMeta(SparkPool01, 8, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, fda6ab23-015f-4d7f-b119-b543a158fb78)

In [16]:
groupedTopPurchases = (sortedTopPurchases.select("visitorId","itemsPurchasedLast12Months")
    .groupBy("visitorId")
    .agg(sum("itemsPurchasedLast12Months").alias("totalItemsPurchased"))
    .orderBy("visitorId") )

display(groupedTopPurchases.limit(100))

StatementMeta(SparkPool01, 8, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, 68368cc7-5a30-4664-b8ee-63de4c82b74c)

In [17]:
topPurchases.createOrReplaceTempView("top_purchases")

StatementMeta(SparkPool01, 8, 18, Finished, Available)

In [19]:
%%spark
// Make sure the name of the dedcated SQL pool (SQLPool01 below) matches the name of your SQL pool.
val df = spark.sqlContext.sql("select * from top_purchases")
df.write.synapsesql("SQLPool01.wwi.TopPurchases", Constants.INTERNAL)

StatementMeta(SparkPool01, 8, 21, Finished, Available)

df: org.apache.spark.sql.DataFrame = [visitorId: bigint, productId: bigint ... 1 more field]


In [20]:
dfsales = spark.read.load('abfss://wwi-02@' + datalake + '.dfs.core.windows.net/sale-small/Year=2019/Quarter=Q4/Month=12/*/*.parquet', format='parquet')

StatementMeta(SparkPool01, 8, 22, Finished, Available)

In [21]:
%%spark
// Make sure the name of the SQL pool (SQLPool01 below) matches the name of your SQL pool.
val df2 = spark.read.synapsesql("SQLPool01.wwi.TopPurchases")
df2.createTempView("top_purchases_sql")

df2.head(10)

StatementMeta(SparkPool01, 8, 23, Finished, Available)

df2: org.apache.spark.sql.DataFrame = [visitorId: bigint, productId: bigint ... 1 more field]
res9: Array[org.apache.spark.sql.Row] = Array([107728,4861,77], [102300,3177,11], [107728,2939,43], [102300,1001,49], [107728,2705,13], [102300,3553,83], [107729,219,42], [102300,3576,74], [107729,4926,11], [102300,1646,74])


In [22]:
dfTopPurchasesFromSql = sqlContext.table("top_purchases_sql")

StatementMeta(SparkPool01, 8, 24, Finished, Available)

In [23]:
inner_join = dfsales.join(dfTopPurchasesFromSql,
    (dfsales.CustomerId == dfTopPurchasesFromSql.visitorId) & (dfsales.ProductId == dfTopPurchasesFromSql.productId))

inner_join_agg = (inner_join.select("CustomerId","TotalAmount","Quantity","itemsPurchasedLast12Months","top_purchases_sql.productId")
    .groupBy(["CustomerId","top_purchases_sql.productId"])
    .agg(
        sum("TotalAmount").alias("TotalAmountDecember"),
        sum("Quantity").alias("TotalQuantityDecember"),
        sum("itemsPurchasedLast12Months").alias("TotalItemsPurchasedLast12Months"))
    .orderBy("CustomerId") )

display(inner_join_agg.limit(100))

StatementMeta(SparkPool01, 8, 25, Finished, Available)

SynapseWidget(Synapse.DataFrame, c6702ac1-dc46-4882-8745-6e71d8e4480e)